In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from collections import  Counter
plt.style.use('ggplot')
import re
import sklearn.metrics 
from nltk.tokenize import word_tokenize
import gensim
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.models import load_model
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,Dropout
from keras.initializers import Constant
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam

In [2]:
#modelbin = load_model('my_modelbin.h5')
#modelplu = load_model('my_modelplu.h5')
print(tf.__version__)

2.3.0


In [3]:
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
import re
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")
english_stopwords = stopwords.words("english")

def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    tokens = [token for token in tokens if token not in english_stopwords]

              

    text = " ".join(tokens)
    pattern3= r"[\d]"
    pattern2="[.]"
    text=re.sub(pattern3, "", text)
    text=re.sub(pattern2, "", text)
    if(text!='') and len(text)>3:
        return text
    else:
        return "#VALID!"

In [4]:
from tensorflow.python.keras.preprocessing import sequence
from tensorflow.python.keras.preprocessing import text
TOP_K = 20000
EMBEDDING_VECTOR_LENGTH = 50 # <=200
MAX_SEQUENCE_LENGTH = 50

class CustomTokenizer:
    def __init__(self, train_texts):
        self.train_texts = train_texts
        self.tokenizer = Tokenizer(num_words=TOP_K)
    def train_tokenize(self):
        # Get max sequence length.
        max_length = len(max(self.train_texts , key=len))
        print(max_length)
        self.max_length = min(max_length, MAX_SEQUENCE_LENGTH)
    
        # Create vocabulary with training texts.
        self.tokenizer.fit_on_texts(self.train_texts)
        
    def vectorize_input(self, tweets):
        # Vectorize training and validation texts.
        
        tweets = self.tokenizer.texts_to_sequences(tweets)
        # Fix sequence length to max value. Sequences shorter than the length are
        # padded in the beginning and sequences longer are truncated
        # at the beginning.
        tweets = sequence.pad_sequences(tweets, maxlen=self.max_length, truncating='post',padding='post')
        return tweets

In [5]:
modelbin = load_model('my_modelbin.h5')
modelplu = load_model('my_modelplu.h5')

In [6]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pickle
from tensorflow.python.keras.preprocessing import sequence
from tensorflow.python.keras.preprocessing import text
text_to_display = widgets.Text()
display(text_to_display)
text = []
def Ppredictbin(inpt):
    inp = []
    for i in inpt:
        inp.append(preprocess_text(i))
    print(inp)
    inn =[]
    inn.append(inp.pop())
    #print(inn)
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)
    tokenized_inpt = tokenizer.vectorize_input(inn)


    #print(tokenized_inpt)
    score = modelbin.predict(tokenized_inpt)
    outpt = max(np.round(score).astype(int))
    outscore = max(score)
    print(outpt[0], outscore[0])


def Ppredictplu(inpt):
    mapa={0:"Анализ данных",1:"Провел встречу",2:"Подготовил отчет",3:"Разработал функциональность",4:"Сделал документацию",5:"Развертывание сервера",6:"Отпуск",7:"Обучение"}
    inp = []
    for i in inpt:
        inp.append(preprocess_text(i))
    #print(inp)
    inn =[]
    inn.append(inp.pop())
    #print(inn)
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)
    tokenized_inpt = tokenizer.vectorize_input(inn)
    print(tokenized_inpt)
    scoreplu = modelplu.predict(tokenized_inpt)
    #print(score[0])
    maxarr = []
    maxarrout = []
    maxarval=[]
    for i in range(len(scoreplu)):
        maxarr.append([np.argmax(scoreplu[i])])
        maxarval.append(max(scoreplu[i]))
    for i in maxarr:
        maxarrout.append(i[0])
    outpt = []
    outpt.append(mapa[maxarrout[np.argmax(maxarval)]])
    print(outpt[0],max(maxarval))
def on_button_clicked(b):
    with out:
        clear_output()
        text.append(text_to_display.value)
        print(Ppredictbin(text),Ppredictplu(text))

b = widgets.Button(
    description='Классификация',
    disabled=False,
    button_style='info',
    tooltip='Click me',
    icon='check'
)
display(b)

out = widgets.Output()
display(out)

b.on_click(on_button_clicked)

text = []

Text(value='')

Button(button_style='info', description='Классификация', icon='check', style=ButtonStyle(), tooltip='Click me'…

Output()